# Train Recommender System

#### Refereces
* https://blog.fastforwardlabs.com/2018/04/10/pytorch-for-recommenders-101.html
* https://medium.com/@iliazaitsev/how-to-implement-a-recommendation-system-with-deep-learning-and-pytorch-2d40476590f9
* https://towardsdatascience.com/recommender-systems-using-deep-learning-in-pytorch-from-scratch-f661b8f391d7
* http://hameddaily.blogspot.com/2016/12/simple-matrix-factorization-with.html?m=1

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np

import models
from movie100k_dataset_np import MovieLensNumpy
import utils_train
print("PyTorch Version: ",torch.__version__)

# Hyperparameters
num_epochs = 50
batch_size = 100
base_lr = 1e-2
n_factors = 20
num_users = 944
num_movies = 1683

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
if device != 'cpu':
    num_gpu = torch.cuda.device_count()
    batch_size *= num_gpu
    base_lr *= num_gpu
    print('Number of GPUs Available:', num_gpu)

PyTorch Version:  1.1.0
Device: cuda:0
Number of GPUs Available: 8


#### Load Train/Validation data

In [2]:
train_dataset = MovieLensNumpy('train.npy')
val_dataset = MovieLensNumpy('val.npy')

# Create training and validation dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16)
dataloaders_dict = {'train': train_loader, 'val': val_loader}

#### Initialize Tensorboard

In [3]:
# Default directory "runs"
writer = SummaryWriter()

#### Initialize Model

In [4]:
user_dummy = torch.randint(0, num_users, (10,))
movie_dummy = torch.randint(0, num_movies, (10,))
#recommender_model = models.MatrixFactorization(num_users, num_movies, n_factors=n_factors)
recommender_model = models.DenseNet(num_users, num_movies, n_factors=n_factors)

# Add on Tensorboard the Model Graph
writer.add_graph(recommender_model, [user_dummy, movie_dummy])

/mnt/anaconda3/lib/python3.7/site-packages/torch/onnx/symbolic.py:569: UserWarning: ONNX export squeeze with negative axis -1 might cause the onnx model to be incorrect. Negative axis is not supported in ONNX. Axis is converted to 1 based on input shape at export time. Passing an tensor of different rank in execution will be incorrect.
  "Passing an tensor of different rank in execution will be incorrect.")


#### Distribute model to multiple GPUs if available

In [5]:
if num_gpu > 1:
    recommender_model = nn.DataParallel(recommender_model)
recommender_model = recommender_model.to(device)

#### Initialize Optimizer and Loss Function

In [6]:
loss_fn = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(recommender_model.parameters(),lr=base_lr)
sc_plt = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)    

#### Train

In [ ]:
best_model, _ = utils_train.train_model(recommender_model, dataloaders_dict, loss_fn, optimizer, sc_plt, writer, device, num_epochs=num_epochs)
torch.save(best_model.state_dict(), './best_model.pkl')

Epoch 0/49
----------
train Loss: 0.0019
val Loss: 0.0016
Epoch 1/49
----------
train Loss: 0.0016
val Loss: 0.0015
Epoch 2/49
----------
train Loss: 0.0015
val Loss: 0.0015
Epoch 3/49
----------
train Loss: 0.0015
val Loss: 0.0015
Epoch 4/49
----------
train Loss: 0.0015
val Loss: 0.0015
Epoch 5/49
----------
train Loss: 0.0014
val Loss: 0.0015
Epoch 6/49
----------
train Loss: 0.0014
val Loss: 0.0015
Epoch 7/49
----------
train Loss: 0.0014
val Loss: 0.0015
Epoch 8/49
----------
train Loss: 0.0014
val Loss: 0.0015
Epoch 9/49
----------
train Loss: 0.0014
val Loss: 0.0014
Epoch 10/49
----------
train Loss: 0.0014
val Loss: 0.0014
Epoch 11/49
----------
